# Apply Warp Transform

## Library Imports

In [1]:
%%time
# Standard Library Imports
import os

# Third Party Imports
import tifffile
import ants
import numpy as np

# Local Imports
import clearex
from tests import download_test_registration_data


CPU times: user 12.5 s, sys: 486 ms, total: 13 s
Wall time: 3.32 s


## Download Example Data

In [2]:
%%time
output_path = download_test_registration_data()

Test data already exists at /endosome/archive/bioinformatics/Danuser_lab/Dean/dean/git/clearex/downloaded_data. Skipping download.
CPU times: user 956 μs, sys: 0 ns, total: 956 μs
Wall time: 1.41 ms


## Specify Paths

In [5]:
%%time

fixed_path = os.path.join(
    output_path,
    "cropped_fixed.tif")

moving_path = os.path.join(
    output_path,
    "cropped_moving.tif")

transform_path = os.path.join(
    output_path,
    "GenericAffine.mat")

transform_list = [transform_path]

registered_path = os.path.join(
    output_path,
    "registered.tif")

CPU times: user 18 μs, sys: 0 ns, total: 18 μs
Wall time: 32.2 μs


## Load Images and Transform

In [6]:
%%time
fixed = tifffile.imread(fixed_path)
fixed = ants.from_numpy(fixed)

moving = tifffile.imread(moving_path)
min_value = moving.min()
max_value = moving.max()
moving = ants.from_numpy(moving)


CPU times: user 404 ms, sys: 181 ms, total: 585 ms
Wall time: 804 ms


## Apply Transform

In [7]:
%%time
warped_image = ants.apply_transforms(
    fixed=fixed,
    moving=moving,
    transformlist=transform_list,
    interpolator='linear'
)

CPU times: user 1.08 s, sys: 305 ms, total: 1.39 s
Wall time: 1.62 s


## Get Information About the Transformed Image

In [8]:
%%time
warped_image

CPU times: user 7 μs, sys: 1 μs, total: 8 μs
Wall time: 15.3 μs


ANTsImage (RAI)
	 Pixel Type : unsigned int (uint32)
	 Components : 1
	 Dimensions : (925, 163, 306)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

## Export the Image

In [9]:
warped_image = warped_image.numpy().astype(np.uint16)

# Export the transformed and cropped image
clearex.registration.common.export_tiff(
    image=warped_image,
    min_value=min_value,
    max_value=max_value,
    data_path=registered_path)

In [10]:
print(f"Data exported to {registered_path}")

Data exported to /endosome/archive/bioinformatics/Danuser_lab/Dean/dean/git/clearex/downloaded_data/registered.tif
